In [1]:
from util.data import createDummyDataset

createDummyDataset(10000,8)

In [ ]:
from models.GDN import GDN
from util.consts import Tasks
from models.BaseMode import BaseModel


In [ ]:
# Example tensor
tensor = torch.tensor([0.5, 1.2, 3.7, 0.9, 2.5])

# Define threshold and value R
threshold = 1.0
R = -1

# Apply the mapping
new_tensor = torch.where(tensor > threshold, torch.tensor(1), torch.tensor(R))

print(new_tensor)

In [ ]:
x=torch.arange(1,11).float()
y=x+1
l=torch.nn.MSELoss(reduce=False)
l(x,y)>1

In [ ]:
feature=4

a=[]
b=[]
for i in range(feature):
    for j in range(feature):
        if i!=j:
            a.append(i)
            b.append(j)
edges=[torch.tensor([a,b])]

In [ ]:
m = GDN(
    edge_index_sets=edges,topk=2, node_num=feature, task=Tasks.next_label
)

In [ ]:

x = torch.rand([10, feature, 3])
m(x, None)

In [ ]:
l=torch.arange(25).reshape(5,-1)
print(l)
l[:,0:3]

In [ ]:

# Number of unique items and the size of each embedding vector
num_items = 10  # Assume we have 10 unique items
embedding_dim = 4  # Each item will be represented as a 4-dimensional vector

# Define the embedding layer
w=torch.randn([num_items, embedding_dim])
w


In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.l=torch.nn.Linear(5,6)
        self.w=torch.nn.Parameter(
            torch.ones([3,6])
        )
    def forward(self,x):
        x=self.l(x)
        x=self.w.matmul(x)
        return x
        

In [ ]:
model=Model()
for p in model.parameters():
    print(p.shape)
model(torch.rand(5))

In [ ]:

embedding_layer = torch.nn.Embedding(num_items, embedding_dim,_weight=w)


In [ ]:
a=torch.rand(8)
print(a)
dp = torch.nn.Dropout(0.5)
dp(a)

In [ ]:
w[5]

In [ ]:

# Example item IDs (e.g., items bought by a user)
item_ids = torch.tensor([2, 5, 7], dtype=torch.long)

# Get the embeddings for the given item IDs
item_embeddings = embedding_layer(item_ids)

print("Item IDs:", item_ids)
print("Corresponding Embeddings:\n", item_embeddings)


In [ ]:
bn=torch.nn.BatchNorm1d(embedding_dim)
out:torch.Tensor=bn(item_embeddings)
out

In [ ]:
bn(torch.randn(3, embedding_dim))